#NPC Chatbot App with ChatGPT, LangChain and Streamlit

#Install Dependencies

In [ ]:
# Install Dependencies
!pip install langchain==0.1.12
!pip install langchain-openai==0.0.8
!pip install langchain-community==0.0.29
!pip install streamlit==1.32.2
!pip install chromadb==0.4.24
!pip install pyngrok==7.1.5
!pip install nltk

#Load OpenAI API Credentials

In [ ]:
# Load OpenAI API Credentials
from getpass import getpass


OPENAI_KEY = getpass('Enter Open AI API Key: ')


#Set Environment Variable

In [ ]:
import os
os.environ['OPENAI_API_KEY'] = OPENAI_KEY

#Write App Code Header

In [ ]:
%%writefile app.py
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_community.chat_message_histories import StreamlitChatMessageHistory
from langchain.callbacks.base import BaseCallbackHandler
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores.chroma import Chroma
from langchain.chains import LLMChain
from operator import itemgetter

import streamlit as st
import tempfile
import os
import nltk
from nltk.tokenize import sent_tokenize

nltk.download('punkt')

# Customize initial app landing page
st.set_page_config(page_title="NPC Simulation Chatbot", page_icon="🎭")
st.title("NPC Simulation Chatbot 🎭")
st.sidebar.header("NPC Character Document Upload")

@st.cache_resource(ttl="1h")
def configure_npc_agent(uploaded_file):
    if uploaded_file is not None:
        text_content = uploaded_file.read().decode("utf-8")

        # --- Normalize line endings to standard Unix-style (\n) ---
        text_content = text_content.replace('\\r\\n', '\\n').replace('\\r', '\\n')

        # --- SECTION PARSING: Line-based parsing ---
        lines = text_content.strip().split('\n') # Split into lines
        sections_dict = {}
        current_section_name = None
        current_section_content = []

        for line in lines:
            line = line.strip()
            if line.startswith("---") and line.endswith("---"):
                if current_section_name:
                    sections_dict[current_section_name] = "\n".join(current_section_content).strip()
                    current_section_content = [] # Reset content for new section
                current_section_name = line.strip('-').strip() # Extract section name
            elif current_section_name:
                current_section_content.append(line) # Append line to current section

        if current_section_name and current_section_content: # Capture last section
            sections_dict[current_section_name] = "\n".join(current_section_content).strip()


        character_description = sections_dict.get("Character Description", "")
        backstory_world_info = sections_dict.get("Backstory and World Info", "")
        quests_text = sections_dict.get("Quests", "")

        quests = [q.strip() for q in quests_text.strip().split("- ") if q.strip()]

        return character_description, backstory_world_info, quests

    return None, None, None


# Manages live updates to a Streamlit app's display by appending new text tokens
class StreamHandler(BaseCallbackHandler):
    def __init__(self, container, initial_text=""):
        self.container = container
        self.text = initial_text

    def on_llm_new_token(self, token: str, **kwargs) -> None:
        self.text += token
        self.container.markdown(self.text)

# UI element to upload text document
uploaded_file = st.sidebar.file_uploader(
    label="Upload NPC Character Text Document", type=["txt"]
)

if uploaded_file is None:
    st.info("Please upload your NPC Character Text Document to continue.")
    st.stop()

# Configure NPC agent
character_description, backstory_world_info, quests = configure_npc_agent(uploaded_file)

if character_description is None or backstory_world_info is None or quests is None:
    st.error("Error loading character document. Please check the format.")
    st.stop()

# Split backstory and world info into chunks for RAG
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_overlap=200,
    chunk_size=2000,
    length_function=len,
)
doc_chunks = text_splitter.create_documents([backstory_world_info])

# Create embeddings and vectorstore for backstory/world info
embeddings_model = OpenAIEmbeddings()
vectordb = Chroma.from_documents(doc_chunks, embeddings_model)
retriever = vectordb.as_retriever()


# Load LLM
chatgpt = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.7, streaming=True)

# System prompt for NPC character
system_prompt_template = SystemMessagePromptTemplate.from_template(character_description)

# RAG Prompt for retrieving NPC/World info
rag_prompt_template = ChatPromptTemplate.from_messages([
    system_prompt_template,
    HumanMessagePromptTemplate.from_template("""Use the following context to answer the user question about yourself or the world.
                                              Context: {context}
                                              Question: {question}
                                              If you cannot answer from the context, just say you do not know.
                                              Answer in character as described in the system prompt.""")
])


# General conversation prompt (no RAG)
conversation_prompt_template = ChatPromptTemplate.from_messages([
    system_prompt_template,
    HumanMessagePromptTemplate.from_template("{question}")
])


# Quest related prompts
quest_suggestion_prompt_template = ChatPromptTemplate.from_messages([
    system_prompt_template,
    HumanMessagePromptTemplate.from_template("Suggest a quest for the user, keeping in character. If no quests are appropriate, just say 'No quests available right now.'")
])
quest_suggestion_chain = quest_suggestion_prompt_template | chatgpt


def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

rag_chain = (
    {
        "context": itemgetter("question") | retriever | format_docs,
        "question": itemgetter("question")
    }
    | rag_prompt_template
    | chatgpt
)
conversation_chain = conversation_prompt_template | chatgpt


# --- Question Type Classifier Chain for NPC Chatbot ---
npc_question_type_prompt_template = """
Determine the type of question being asked to the NPC. Choose from the following categories: quest_request, info_request, or general_conversation.

Question: {question}

Respond with 'quest_request' if the question is clearly asking for a quest, or about quests in general (e.g., "give me a quest", "offer a quest", "do you have quests?", "what quests are available?", "i want a quest", "tell me about quests", "any tasks for me?").
Respond with 'info_request' if the question is asking for information about the NPC or their world, requiring context from their background (e.g., "tell me about yourself", "who are you?", "where are we?", "what is this place?", "tell me about the world").
Respond with 'general_conversation' if the question is just a general greeting, statement, or question that doesn't clearly fall into 'quest_request' or 'info_request' (e.g., "hello", "how are you?", "what do you do?", "nice weather today").

Just answer 'quest_request', 'info_request', or 'general_conversation'.
"""
npc_question_type_prompt = ChatPromptTemplate.from_template(npc_question_type_prompt_template)
npc_question_type_chain = npc_question_type_prompt | chatgpt
# --- End Question Type Classifier Chain ---


# Conversation Handling
streamlit_msg_history = StreamlitChatMessageHistory(key="npc_chat_messages")

if len(streamlit_msg_history.messages) == 0:
    streamlit_msg_history.add_ai_message("Greetings! How can I help you today?")

for msg in streamlit_msg_history.messages:
    st.chat_message(msg.type).write(msg.content)

if user_prompt := st.chat_input():
    st.chat_message("human").write(user_prompt)

    # --- Classify question type ---
    question_type_response = npc_question_type_chain.invoke({"question": user_prompt})
    question_type = question_type_response.content.strip()
    st.write(f"--- **DEBUGGING: Question Type:** --- {question_type}") # Debugging question type

    if question_type == "quest_request":
        # --- DEBUGGING: Available quests on quest_request ---
        st.write(f"--- **DEBUGGING: AVAILABLE QUESTS ON USER QUEST REQUEST** ---")
        st.write(f"Quests list at start of quest handling: {quests}")
        st.write(f"--- **END DEBUGGING: AVAILABLE QUESTS** ---")
        # --- END DEBUGGING ---

        if quests:
            quest_to_offer = quests.pop(0) # Get the first quest and remove it from the list

            # --- Direct Quest Output - Bypassing LLM for Quest Text ---
            with st.chat_message("ai"):
                # Fixed, in-character intro (outside LLM)
                quest_intro = "Right then, listen up! Here's your task:"  # Customize this intro

                # Directly output the quest text using st.markdown for formatting
                quest_message = f"{quest_intro}\n\n**{quest_to_offer}**"
                st.markdown(quest_message)

            streamlit_msg_history.add_user_message(user_prompt)
            streamlit_msg_history.add_ai_message(quest_message)
            # --- End Direct Quest Output ---

        else:
             with st.chat_message("ai"):
                stream_handler = StreamHandler(st.empty())
                config = {"callbacks": [stream_handler]}
                response = quest_suggestion_chain.invoke({}, config)
             streamlit_msg_history.add_user_message(user_prompt)
             streamlit_msg_history.add_ai_message(response.content)


    elif question_type == "info_request":
        # RAG for NPC/World info
        with st.chat_message("ai"):
            stream_handler = StreamHandler(st.empty())
            config = {"callbacks": [stream_handler]}
            response = rag_chain.invoke({"question": user_prompt}, config)
        streamlit_msg_history.add_user_message(user_prompt)
        streamlit_msg_history.add_ai_message(response.content)

    elif question_type == "general_conversation":
        # General conversation
        with st.chat_message("ai"):
            stream_handler = StreamHandler(st.empty())
            config = {"callbacks": [stream_handler]}
            response = conversation_chain.invoke({"question": user_prompt}, config)
        streamlit_msg_history.add_user_message(user_prompt)
        streamlit_msg_history.add_ai_message(response.content)

    else: # Fallback, should ideally not reach here if classification is comprehensive
        with st.chat_message("ai"):
            stream_handler = StreamHandler(st.empty())
            config = {"callbacks": [stream_handler]}
            response = conversation_chain.invoke({"question": user_prompt}, config)
        streamlit_msg_history.add_user_message(user_prompt)
        streamlit_msg_history.add_ai_message(response.content)

#Starting the Streamlit App

In [ ]:
# Starting the Streamlit App
!streamlit run app.py --server.port=8989 &>/./logs.txt &

#Setting Up ngrok Tunnel

In [ ]:
# Setting Up ngrok Tunnel
from getpass import getpass

ngrok_auth_token = getpass('Enter ngrok API Key: ')

In [ ]:
from pyngrok import ngrok
import yaml

# Terminate open tunnels if exist
ngrok.kill()

# Authenticate ngrok with the token read from the file
!ngrok config add-authtoken {ngrok_auth_token}

# Open an HTTPS tunnel on port XXXX which you get from your `logs.txt` file
ngrok_tunnel = ngrok.connect(8989)
print("Streamlit App:", ngrok_tunnel.public_url)